# 💡 How2DDDA: Buckingham Pi Theorem & Dimensional Analysis

> All code and examples are shared to help researchers, students, and engineers understand the reasoning behind DDDA — and to make it easy to apply dimensional analysis to your own data.  
> This notebook serves as an entry-level guide for teaching, validating physical models, and enabling domain-specific knowledge engineering through data-driven dimensional reasoning.

---

## 🎯 What You'll Learn

In this notebook, we will walk through the theoretical and computational foundation of **dimensional analysis**, with a focus on the **Buckingham Pi theorem**. You will learn:

1. **The motivation behind dimensional analysis**  
   Understand why we reduce variables and how dimensional consistency enables model generalization.

2. **How to construct the dimensional matrix (D-matrix)**  
   Encode physical units of input quantities using base units and build the D-matrix.

3. **How to compute Π-groups using null space techniques**  
   Discover dimensionless groups by solving linear algebraic equations on the D-matrix.

4. **How to interpret and validate Π-groups**  
   Learn to assess whether derived groups make physical and computational sense.

5. **How to extend the method toward data-driven workflows**  
   Set the stage for further steps in the DDDA pipeline including Pi-group selection, uncertainty quantification, and regime detection.

---

## 👤 Author

- **Name**: Jiashun Pang  
- **Created**: August 2025  
- **Affiliation**: DDDA Project, open research notebook  
- **Notebook Focus**:  
  A hands-on exploration of dimensional analysis — from aggregated raw quantities to symbolic Pi-group discovery and preparation for downstream DDDA tasks.

---

📌 *This notebook is designed to be accessible for learners new to dimensional analysis, while also laying the foundation for advanced applications in the full DDDA pipeline.*


# 1. **The motivation behind dimensional analysis**  
The most general form of physical equations中我们使用隐函数来表达一个函数关系

$f(x_1, x_2,...,x_n)=0$   (1)

- 存在一个（可能未知）函数关系，
- 不一定能显式解出 $x_1=f(x_2,...x_3)$
- 但变量之间一定满足某个不变关系或约束。

如果$x_n$之间存在关系，我们可以使用ratio来表示。 那我们重构一下(1)，其中使用$Q_n$作为物理量，$r$作为物理量的关系，也就是ratio

$f(Q_1, Q_2,...,Q_n, r', r'',...)=0$   (2)

接下来我们深入一步，来了解'Complete equation'

> If the quantities Q are measured by an absolute system of units,
> the coefficients of the equation do not depend on the sizes of the fundamental units,
> but only on the fixed interrelations of the units which characterize the system and differentiate it from any other absolute system.

其实很简单——我们要描述equation中所有的关系。最终维度一致，且使得系数为无量纲数。

✅ 物理规律中那些有“物理意义”的系数必须是无量纲的、与单位大小无关；

✅ 即使我们在不同单位体系中工作，只要它们都是“绝对单位制”，这些规律依然保持不变；

✅ 真正发生变化的，可能只是单位间的定义结构（interrelations），但不是系数本身。

举个complete equation的例子 - 等温过程：

$\frac{\rho\nu}{\theta} = constant$ (3)

其中$\rho$ 是密度，$\nu$ 是specfic volume, $\theta$ 是absolute temperature of a mass of gas

(3)不是complete equation，因为其中的constant跟选择的气体以及$\rho, \nu, \theta$的测量单位相关。

但是，如果更进一步挖掘，引入气体常数：

$\frac{\rho\nu}{\theta R} = N$ (4)

这样:

- N 不依赖于单位的绝对“大小”；

- 它只取决于你选用的单位体系中，R 如何通过 ρ,ν,θ 表达；

- 一旦这个“单位系统”固定了，N 就是一个不变的纯数。

✅ 结论：我们得到了一个维度齐次、单位无关、结构完整的完整方程 (complete equation)。

接下来，从逻辑上来理解。

> ❗ 物理方程中，只能使用带有单位的量参与乘幂与乘除，而不能将带单位的量作为 log、sin、exp 等“纯数学函数”的输入。

我们来举例说明：

$\sum M Q_1^{b_1} Q_2^{b_2}...=0$

- m 为单位米，$\log(2m)$没有意义，而$\log(\frac{2m}{1m})=\log(2)$有意义，因为这个是无量纲比值。除了乘法(如2Q)可以直接对带单位的量进行操作，其他运算($\log ,\sin ,\cos ,\tan ,\exp$等)都只能对'纯数'(无量纲量)操作。这是因为：这些数学函数的输出没有单位，因此输入必须也无单位，才能保持物理一致性。
- 

> 纯数学函数的输入必须无量纲，否则无法保证级数展开过程中项与项可相加，物理维度亦将失衡。而幂函数具有清晰的单位含义所以可以成立，如长度l有单位m, 我们可以写$l^2$, 它表示的是面积A.

> 物理方程的每一项都应该是多个带单位量的乘积形式（monomial），且每一项具有物理单位，并在等式中维度一致，最终构成complete equation。

- monomial（单项式） 是指$M=c\cdot x_1^{a_1} x_2^{a_2}\cdot \cdot \cdot x_n^{a_n}$
- 每项必须“维度一致”
- 所有变量只能通过“乘幂 + 相乘”组合
- 不能混加不同维度的项

最终根据Buckingham说：

>一个合理的物理方程，其结构形式只能是若干个 monomial 项的线性组合：

$$
f(Q_1, Q_2,...,Q_n)=M_1 Q_1^{a_11} Q_2^{a_12}...+M_2 Q_1^{a_21} Q_2^{a_22}...+...
$$

这个就是：

$$
f(Q_1, Q_2,\ldots,Q_n) = \sum_{i=1}^{m} M_i \prod_{j=1}^{n} Q_j^{a_{ij}}
$$

- 每一项$M_i Q_1^{a_{i1}} Q_2^{a_{i2}}$是一个 monomial（乘积项）；

- 整个物理方程是这些 monomials 的线性组合（即 求和）。

✅ 总结
| 项                        | 是否要求维度一致？ | 例子                                |
| ------------------------ | --------- | --------------------------------- |
| **monomial 内部**（变量相乘）    | ❌ 不要求维度相同 | $\rho v^2$, $\mu \frac{v}{L}$     |
| **monomial 之间求和 $\sum$** | ✅ 必须维度相同  | $\rho v^2 + \mu \frac{v}{L}$（压力项） |
| **Π 群之间用作函数参数**          | ✅ 都需无量纲   | $F(\Pi_1, \Pi_2) = 0$             |


# 1 · The Motivation Behind Dimensional Analysis

> "Dimensional analysis is but an aid to *seeing* the essentials of a physical problem. —E. Buckingham (1914)"

---

## 1.1 Implicit Functional Form

A **general physical law** can be written as an implicit relation

$$
f(x_1, x_2, \dots, x_n)=0 \tag{1}
$$

* 变量间存在某种（可能未知）函数关系；
* 无需解析求出 \$x\_1 = f(x\_2,\dots)\$；
* 但它们必须满足同一约束——此约束正是我们要提炼的“物理规律”。

---

## 1.2 将变量重写为“比值”

当若干 \$x\_i\$ 之间存在 **相似尺度** 时，可引入无量纲比值（ratios）\$r', r'',\ldots\$，并用物理量 \$Q\_i\$ 表示变量：

$$
f(Q_1, Q_2, \dots, Q_n, r', r'', \dots)=0. \tag{2}
$$

这些 ratio 预示了变数可进一步压缩为 **Π 群**。

---

## 1.3 Complete Equation 的概念

> *If the quantities \$Q\$ are measured in an absolute system of units, the coefficients of the equation do not depend on the sizes of the fundamental units, but only on the fixed inter‑relations of the units which characterize the system.* —Buckingham

**含义提炼**

| 结论          | 含义                       |
| ----------- | ------------------------ |
| 系数必须无量纲     | 物理方程在任何单位制下均保持不变         |
| 单位改变 ≠ 系数改变 | 真正变化的是各单位之间的*定义关系*，非系数本身 |

### 例 · 等温过程

初式

$$
\frac{\rho\,\nu}{\theta}=\text{constant}\tag{3}
$$

因 *constant* 随所选气体及单位而变，(3) **不是** Complete Equation。

引入气体常数 \$R\$ 得

$$
\frac{\rho\,\nu}{\theta R}=N, \tag{4}
$$

其中 \$N\$ 为 **纯数**，与单位大小无关 → (4) 即 **Complete Equation**（维度齐次、单位无关、结构完整）。

---

## 1.4 物理方程允许的算子

**定理** ：物理方程中只允许“乘、除、幂”作用于带单位量；若用到 \$\log,\sin,\exp\$ 等**纯数学函数**，其自变量必须先无量纲化。

### 1.4.1 为什么？

| 级数展开视角                                | 维度冲突                     |
| ------------------------------------- | ------------------------ |
| \$e^{x}=1+x+\tfrac{x^{2}}{2!}+\dots\$ | 若 \$x\$ 带单位，则各项单位不同，无法相加 |
| \$\sin x=x-\tfrac{x^{3}}{3!}+\dots\$  | 同理，必须 \$x\$ 为纯数          |

### 1.4.2 幂函数为何可行？

* \$l\$ 有单位 *m*，\$l^{2}\$ 表示面积 (\$\text{m}^{2}\$)，物理含义明确。

---

## 1.5 “单项式 (mononomial) + 线性组合” 结构

一个合法物理方程可写成多个 **monomial** 之和：

$$
f(Q_1, \dots,Q_n)=\sum_{i=1}^{m}M_i\prod_{j=1}^{n}Q_j^{a_{ij}},\tag{5}
$$

其中每项 \$M\_i\prod Q\_j^{a\_{ij}}\$ 均为乘积式 **monomial**。

> ✧ **加法要求维度一致**；monomial 内部则通过乘幂自行生成所需维度。

### 维度一致性速查表

| 结构                     | 是否必须维度一致 | 示例                          |
| ---------------------- | -------- | --------------------------- |
| monomial 内部 (乘积)       | ✗ 不要求    | \$\rho v^{2}\$, \$\mu v/L\$ |
| monomial 之间求和 \$\sum\$ | ✓ 必须     | \$\rho v^{2}+\mu v/L\$      |
| Π 群作为函数参数              | ✓ 无量纲    | \$F(\Pi\_{1},\Pi\_{2})=0\$  |

---

## 1.6 关键要点回顾

1. **Complete Equation**：维度齐次 + 系数无量纲 + 单位体系无关。
2. **允许算子**：乘、除、幂 → \$Q^{b}\$；log/sin/exp 之前必须无量纲化。
3. **结构范式**：物理方程 = 多个 monomial 的线性组合；Π 理论进一步把它们压缩为无量纲群。

> 这一章奠定了 Π 定理推导的语言和结构约束。下一章我们将正式构建维度矩阵 \$D\$ 并证明“\$n\$ 个变量、\$k\$ 个基本维 → \$n-k\$ 个 Π 群”。


# $\Pi$的引入
### 以量纲视角来看

我们先通过Dimension homogeneous来从
$$\sum M Q_1^{a_1} Q_2^{a_2}...Q_n^{a_n}=0$$

得到：

$$\sum N Q_1^{a_1} Q_2^{a_2} ... + I = 0$$

其中
- N 是dimensionless numbers
- I 为量纲一致规则的归纳量 $[Q_1^{a_1} Q_2^{a_2}...Q_n^{a_n}] = [I]$

由于dimension homogeneous的原因，我们总能通过调整$\alpha$来找到$\Pi$来代表dimensionless product

$$\Pi = Q_1^{a_1} Q_2^{a_2}...Q_n^{a_n}$$

得到：

$$\sum N \Pi + I = 0$$

由于 $\Pi$ 为无量纲，所以 $\Pi^x$ 这种幂形式也符合无量纲条件，所以 $\Pi_1^{x_1} \Pi_2^{x_2} \Pi_3^{x_3} ... \Pi_n^{x_n}$ 也同样是无量纲。

所以得到：

$$\sum N \Pi_1^{x_1} \Pi_2^{x_2} \Pi_3^{x_3} ... \Pi_n^{x_n} + I = 0$$

由此，(2)可以写为如下形式

$$\Psi(\Pi_1, \Pi_2,...\Pi_i)=0,\tag{6}$$



## From 'Dimensionally homogeneous' to 'Dimensionless'

> 任何 dimensionally homogeneous 的物理方程，都可以通过适当选择$\alpha_j$, 把每一项变成若干无量纲组合的乘积或函数

### 以矩阵视角来看

1. 从物理方程到维度矩阵

   - 确保原始方程dimensionally homogeneous(量纲其次)
   - 组合D矩阵： 将每个物理量$Q_j$的基本单位指数$\alpha$排成D矩阵(k行对应基本单位数，n列对应变量数)。

   举例：假设阻力F只由${\rho, U, L, \mu}$决定：

   $$F=f(\rho, U, L, \mu)$$

   基本单位取 M, L, T。其中：


   $$[F] = M^1 L^1 T^{-2},  [\rho] = M^1 L^{-3},  [U] = L^1 T_{-1},  [L] = L^1,  [\mu] = M^1 L^{-1} T^{-1}$$

2. 维度矩阵与“零空间”个数

    组合维度矩阵D：

   $$ D = \begin{pmatrix} 1 & 1 & 0 & 0 & 1\\ 1 & -3 & 1 & 1 & -1 \\ -2 & 0 & -1 & 0 & -1 \end{pmatrix} $$


   变量数 n=5，独立基本单位数 k=3；因此 Π-群个数 i=n−k=2。

    这表示能构成两个相互独立的无量纲组合，方程最终只会写成它们之间的关系。

4. 解$D \cdot a = 0$得到$\Pi$群

5. 得到物理关系的最终形式与含义


